In [1]:
"""
Requirements:
pip install pandas duckduckgo-search rapidfuzz ollama requests
(duckduckgo-search package is sometimes renamed; use `ddgs` if you prefer.)
"""
import sys
!{sys.executable} -m pip install duckduckgo_search rapidfuzz ollama requests



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [2]:
!pip show duckduckgo_search

Name: duckduckgo_search
Version: 8.1.1
Summary: Search for words, documents, images, news, maps and text translation using the DuckDuckGo.com search engine.
Home-page: https://github.com/deedy5/duckduckgo_search
Author: deedy5
Author-email: 
License: MIT License
Location: /Users/sukanyadatta/bm_code/muni_adv_pbf/muni_adv_pbf/lib/python3.12/site-packages
Requires: click, lxml, primp
Required-by: 


In [3]:

import time, json, unicodedata, re, logging, os
from datetime import datetime
import pandas as pd
import requests
from rapidfuzz import fuzz
from duckduckgo_search import DDGS
from ollama import chat  # ollama python package (local Ollama must be running)

import hashlib
import pickle

In [4]:
df = pd.read_csv("https://www.dropbox.com/scl/fi/eqc8ss2ko0gr9pzf71opc/compiled_matches_merge_1to19.csv?rlkey=o99i45umhbh6ruppakqoo7sdn&st=rjxf2e4i&dl=1", index_col=False)
df

,identifier,sale_date,sdc_amt,issuer_sdc,months_to_final_maturity,issue_id_l,issuer_mergent,mergent_amt,offering_date,final_maturity_mergent,...,merge_source,financial_advisor_sdc,financial_advisor_mergent,offering_type_sdc,offering_type_mergent,state,maturity_yr,insured,refunding,issue_description
0,2805558,2003-12-02,19.370,Yorba Linda Public Finance Auth,357.768,100002,YORBA LINDA CALIF LEASE REV CTFS PARTN,19370000.0,2003-12-03,362.733340,...,merge_1,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC,Negotiated-Pub,,CA,362.733340,1.0,1.000000,RV
1,173548,2003-10-15,7.165,Berkeley Co SD,100.104,10002,BERKELEY CNTY S C SCH DIST,7165000.0,2003-10-15,101.500000,...,merge_1,Ross Sinclaire & Associates,NaN,Competitive,,SC,101.500000,0.0,1.000000,GO
2,322361,2011-12-05,3.565,Wilkin (Campbell-Tintah) ISD#852,241.344,1000249,CAMPBELL-TINTAH INDPT SCH DIST NO 852 MINN,3565000.0,2011-12-05,244.700000,...,merge_1,Ehlers & Associates,EHLERS AND ASSOCIATES INC,Competitive,COMP,MN,244.700000,0.0,0.000000,GO
3,295632,2011-11-30,276.800,California Infrstr & Eco Dev Bank,316.212,1000254,CALIFORNIA INFRASTRUCTURE & ECONOMIC DEV BK REV,69200000.0,2011-11-17,320.600010,...,merge_1,Swap Financial Group LLC,NaN,Negotiated-Pub,NEGO,CA,320.600010,0.0,1.000000,RV
4,295633,2011-11-30,276.800,California Infrstr & Eco Dev Bank,316.212,1000255,CALIFORNIA INFRASTRUCTURE & ECONOMIC DEV BK REV,69200000.0,2011-11-17,320.600010,...,merge_1,Swap Financial Group LLC,NaN,Negotiated-Pub,NEGO,CA,320.600010,0.0,1.000000,RV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332132,1389906,2008-12-04,0.650,Macedon Town-New York,12.000,668350,VESTAL N Y CENT SCH DIST,651993.0,2008-12-01,56.799999,...,merge_19,Bernard P Donegan Inc,BERNARD P DONEGAN INC,Competitive,COMP,NY,56.799999,0.0,0.000000,GO
332133,2342444,2007-11-19,38.200,Snohomish Co (Marysville) SD #25,216.060,636272,WASHINGTON ST,37855000.0,2007-11-15,22.333334,...,merge_19,NaN,NaN,Negotiated-Pub,NEGO | PPLC,WA,22.333334,NaN,NaN,GO
332134,2933529,2006-06-16,53.800,Academic Finance Corp,431.604,589820,WHITE SETTLEMENT TEX INDPT SCH DIST,53277592.5,2006-06-15,355.431271,...,merge_19,NaN,SOUTHWEST SECURITIES INC,Negotiated-Pub,NEGO,TX,355.431271,0.0,1.000000,GO
332135,2881868,2002-07-16,9.510,Kane Co Forest Preserve Dt,27.024,98032,WILL & KANKAKEE CNTYS ILL CMNTY UNIT SCH DIST ...,9600000.0,2002-07-15,228.154715,...,merge_19,NaN,NaN,Competitive,,IL,228.154715,1.0,0.093229,GO


In [5]:
df.columns

Index(['identifier', 'sale_date', 'sdc_amt', 'issuer_sdc',
       'months_to_final_maturity', 'issue_id_l', 'issuer_mergent',
       'mergent_amt', 'offering_date', 'final_maturity_mergent',
       'match_status', 'merge_source', 'financial_advisor_sdc',
       'financial_advisor_mergent', 'offering_type_sdc',
       'offering_type_mergent', 'state', 'maturity_yr', 'insured', 'refunding',
       'issue_description'],
      dtype='object')

In [6]:
df2 = df[['identifier','issue_id_l','sale_date','offering_date','financial_advisor_sdc','financial_advisor_mergent']]
df2

,identifier,issue_id_l,sale_date,offering_date,financial_advisor_sdc,financial_advisor_mergent
0,2805558,100002,2003-12-02,2003-12-03,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC
1,173548,10002,2003-10-15,2003-10-15,Ross Sinclaire & Associates,NaN
2,322361,1000249,2011-12-05,2011-12-05,Ehlers & Associates,EHLERS AND ASSOCIATES INC
3,295632,1000254,2011-11-30,2011-11-17,Swap Financial Group LLC,NaN
4,295633,1000255,2011-11-30,2011-11-17,Swap Financial Group LLC,NaN
...,...,...,...,...,...,...
332132,1389906,668350,2008-12-04,2008-12-01,Bernard P Donegan Inc,BERNARD P DONEGAN INC
332133,2342444,636272,2007-11-19,2007-11-15,NaN,NaN
332134,2933529,589820,2006-06-16,2006-06-15,NaN,SOUTHWEST SECURITIES INC
332135,2881868,98032,2002-07-16,2002-07-15,NaN,NaN


In [7]:
# check Nan
cols_to_check = ['financial_advisor_sdc','financial_advisor_mergent']
nan_counts = df[cols_to_check].isna().sum()
nan_counts

financial_advisor_sdc        119230
financial_advisor_mergent    139538
dtype: int64

In [8]:
# check different MA
col1 = df['financial_advisor_sdc'].astype(str).str.lower()
col2 = df['financial_advisor_mergent'].astype(str).str.lower()

match_count = (col1 == col2).sum()
match_count

163855

In [9]:
df_unm1 = df2[(df2['financial_advisor_sdc'].astype(str).str.lower() != df2['financial_advisor_mergent'].astype(str).str.lower())]
df_unm1

,identifier,issue_id_l,sale_date,offering_date,financial_advisor_sdc,financial_advisor_mergent
0,2805558,100002,2003-12-02,2003-12-03,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC
1,173548,10002,2003-10-15,2003-10-15,Ross Sinclaire & Associates,NaN
2,322361,1000249,2011-12-05,2011-12-05,Ehlers & Associates,EHLERS AND ASSOCIATES INC
3,295632,1000254,2011-11-30,2011-11-17,Swap Financial Group LLC,NaN
4,295633,1000255,2011-11-30,2011-11-17,Swap Financial Group LLC,NaN
...,...,...,...,...,...,...
332128,2893081,483099,2002-10-31,2002-10-31,Munistat Services Inc,NaN
332129,2626853,594417,2006-05-10,2006-05-15,Public Finance Associates,NaN
332130,2864136,486444,2001-12-14,2001-12-11,Bernard P Donegan Inc,NaN
332131,2914023,90938,2003-09-22,2003-09-19,NaN,LERCH VINCI AND HIGGINS LLP


In [10]:
df_unm2 = df_unm1[(pd.notna(df_unm1['financial_advisor_sdc'])) & (pd.notna(df_unm1['financial_advisor_mergent']))]
df_unm2

,identifier,issue_id_l,sale_date,offering_date,financial_advisor_sdc,financial_advisor_mergent
0,2805558,100002,2003-12-02,2003-12-03,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC
2,322361,1000249,2011-12-05,2011-12-05,Ehlers & Associates,EHLERS AND ASSOCIATES INC
8,1374110,1000259,2011-12-02,2011-12-02,FirstSouthwest,FIRST SOUTHWEST CO
11,744806,1000266,2011-12-06,2011-12-06,Ehlers & Associates,EHLERS AND ASSOCIATES INC
12,784421,1000267,2011-12-01,2011-12-01,FirstSouthwest,FIRST SOUTHWEST CO
...,...,...,...,...,...,...
332106,1944587,1736370,2021-05-25,2021-06-01,Specialized Public Finance Inc,ESTRADA HINOJOSA AND CO INC
332110,2984526,1788915,2022-06-14,2022-06-07,Ehlers & Associates,ROBERT W BAIRD AND CO INC
332119,268787,1218194,2013-07-15,2013-07-16,FirstSouthwest,FIRST SOUTHWEST CO
332120,2981122,1775090,2021-12-22,2021-12-20,Government Consultants Inc,MUNIGROUP LLC


In [11]:
df_unm3 = df_unm2.drop_duplicates(subset = ['financial_advisor_sdc','financial_advisor_mergent'])
df_unm3

,identifier,issue_id_l,sale_date,offering_date,financial_advisor_sdc,financial_advisor_mergent
0,2805558,100002,2003-12-02,2003-12-03,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC
2,322361,1000249,2011-12-05,2011-12-05,Ehlers & Associates,EHLERS AND ASSOCIATES INC
8,1374110,1000259,2011-12-02,2011-12-02,FirstSouthwest,FIRST SOUTHWEST CO
15,209010,1000281,2011-12-01,2011-12-01,Ross Sinclaire & Associates,ROSS SINCLAIRE AND ASSOCIATES LLC
17,1972312,1000283,2011-11-30,2011-11-30,Umbaugh LLP,H J UMBAUGH AND ASSOCIATES CERTIFIED PUBLIC AC...
...,...,...,...,...,...,...
332097,2583707,1779318,2022-02-02,2022-01-31,Hilltop Securities Inc,HAMLIN CAPITAL ADVISORS LLC
332106,1944587,1736370,2021-05-25,2021-06-01,Specialized Public Finance Inc,ESTRADA HINOJOSA AND CO INC
332110,2984526,1788915,2022-06-14,2022-06-07,Ehlers & Associates,ROBERT W BAIRD AND CO INC
332120,2981122,1775090,2021-12-22,2021-12-20,Government Consultants Inc,MUNIGROUP LLC


In [12]:
df_unm4 = df_unm2
df_unm4['sale_yr'] = pd.to_datetime(df_unm4['sale_date']).dt.year
df_unm5 = df_unm4.drop_duplicates(subset=['financial_advisor_sdc','financial_advisor_mergent','sale_yr'])
df_unm5

/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_1015/2417621085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unm4['sale_yr'] = pd.to_datetime(df_unm4['sale_date']).dt.year


,identifier,issue_id_l,sale_date,offering_date,financial_advisor_sdc,financial_advisor_mergent,sale_yr
0,2805558,100002,2003-12-02,2003-12-03,Harrell & Company Advisors LLC,HARRELL AND CO ADVISORS LLC,2003
2,322361,1000249,2011-12-05,2011-12-05,Ehlers & Associates,EHLERS AND ASSOCIATES INC,2011
8,1374110,1000259,2011-12-02,2011-12-02,FirstSouthwest,FIRST SOUTHWEST CO,2011
15,209010,1000281,2011-12-01,2011-12-01,Ross Sinclaire & Associates,ROSS SINCLAIRE AND ASSOCIATES LLC,2011
17,1972312,1000283,2011-11-30,2011-11-30,Umbaugh LLP,H J UMBAUGH AND ASSOCIATES CERTIFIED PUBLIC AC...,2011
...,...,...,...,...,...,...,...
332098,2881065,63964,2002-07-09,2002-07-12,Capital Markets Advisors LLC,PUBLIC FINANCE ASSOCIATES INC,2002
332106,1944587,1736370,2021-05-25,2021-06-01,Specialized Public Finance Inc,ESTRADA HINOJOSA AND CO INC,2021
332110,2984526,1788915,2022-06-14,2022-06-07,Ehlers & Associates,ROBERT W BAIRD AND CO INC,2022
332120,2981122,1775090,2021-12-22,2021-12-20,Government Consultants Inc,MUNIGROUP LLC,2021


In [13]:
logging.basicConfig(level=logging.INFO)

DDG = DDGS()  # simple sync client


/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_1015/3361049473.py:3: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  DDG = DDGS()  # simple sync client


In [14]:
# Config
INPUT_CSV = "input_names.csv"   # must have columns 'name_a' and 'name_b'
OUTPUT_JSONL = "results_v3.jsonl"
OPENCORP_TOKEN = os.getenv("OPENCORP_TOKEN")  # optional

SUFFIX_TOKENS = {
    "inc","incorporated","corp","corporation","co","company","ltd","limited",
    "llc","llp","plc","gmbh","sa","ag","pte","berhad","nv","spa","sarl","the"
}

DDG_CACHE_FILE = "ddg_cache_v3.pkl"
ddg_cache = {}

# Load cache if exists
if os.path.exists(DDG_CACHE_FILE):
    with open(DDG_CACHE_FILE, "rb") as f:
        ddg_cache = pickle.load(f)

In [16]:
def normalize_name(n: str) -> str:
    if not isinstance(n, str):
        return ""
    s = unicodedata.normalize("NFKD", n).lower()
    s = re.sub(r'[\u2019\u2018]', "'", s)
    s = re.sub(r'[^0-9a-z&\s]', ' ', s)
    s = s.replace('&', ' and ')
    toks = [t for t in s.split() if t.strip()]
    # strip trailing suffix tokens
    while toks and toks[-1] in SUFFIX_TOKENS:
        toks.pop()
    return " ".join(toks).strip()

def fuzzy_scores(n1: str, n2: str) -> dict:
    return {
        "ratio": fuzz.ratio(n1, n2),
        "token_sort": fuzz.token_sort_ratio(n1, n2),
        "token_set": fuzz.token_set_ratio(n1, n2),
        "partial": fuzz.partial_ratio(n1, n2)
    }


In [18]:
# def ddg_search(query: str, max_results: int = 5):
#     try:
#         results = DDG.text(query, max_results=max_results)
#         # results is a list of dicts: { 'title', 'href', 'body' }
#         return results or []
#     except Exception as e:
#         logging.exception("DDG search failed for %s: %s", query, e)
#         return []

def ddg_search(query: str, max_results: int = 5):
        
    h = hashlib.sha256(query.encode()).hexdigest()
    if h in ddg_cache:
        return ddg_cache[h]

    try:
        results = DDG.text(query, max_results=max_results) or []
        ddg_cache[h] = results
        # Save cache to disk
        with open(DDG_CACHE_FILE, "wb") as f:
            pickle.dump(ddg_cache, f)
        return results
    except Exception as e:
        logging.exception("DDG search failed for %s: %s", query, e)
        return []

def opencorp_search(name: str, token: str = None, max_results=3):
    if not token:
        return []  # no token: skip or try unauthenticated endpoint (limited)
    try:
        url = "https://api.opencorporates.com/v0.4/companies/search"
        params = {"q": name, "per_page": max_results, "api_token": token}
        r = requests.get(url, params=params, timeout=10)
        j = r.json()
        return j.get("results", {}).get("companies", [])
    except Exception as e:
        logging.exception("OpenCorporates search failed: %s", e)
        return []


In [19]:
from ollama import embeddings

def semantic_rank(query: str, hits: list, model: str = "nomic-embed-text", top_k: int = 3):
    try:
        # Embed the query
        query_vec = embeddings(model=model, prompt=query)["embedding"]

        def embed_text(text):
            return embeddings(model=model, prompt=text)["embedding"]

        # Score each hit by cosine similarity
        results = []
        for h in hits:
            text = f"{h.get('title', '')} {h.get('body', '')}"
            hit_vec = embed_text(text)
            score = cosine_similarity(query_vec, hit_vec)
            results.append((score, h))

        results.sort(reverse=True, key=lambda x: x[0])
        return [r[1] for r in results[:top_k]]
    except Exception as e:
        logging.warning("Semantic ranking failed: %s", e)
        return hits[:top_k]

def cosine_similarity(vec1, vec2):
    import numpy as np
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))


In [20]:
# LLM prompt builder
SYSTEM_PROMPT = """You are an exacting research assistant. Given two firm names, fuzz scores, and top web hits for each,
decide whether they refer to the same legal entity. Be conservative. Output ONLY valid JSON with
{verdict, confidence (0-100), explanation (brief), evidence_urls (list)}. If uncertain, use verdict 'ambiguous'."""

In [21]:
# def build_user_prompt(original_a, original_b, norm_a, norm_b, scores, hits_a, hits_b, opencorp_a, opencorp_b):
def build_user_prompt(original_a, original_b, norm_a, norm_b, scores, summary_a, summary_b, opencorp_a, opencorp_b):
    def hits_text(hits):
        lines = []
        for i, h in enumerate(hits[:3]):
            title = h.get("title") or ""
            href = h.get("href") or h.get("url") or ""
            body = h.get("body") or ""
            lines.append(f"{i+1}. {title} | {href} | {body[:300]}")
        return "\n".join(lines) if lines else "(no hits)"
    oc_text = ""
    if opencorp_a or opencorp_b:
        oc_text += "OpenCorporates A: " + json.dumps(opencorp_a[:2], default=str) + "\n"
        oc_text += "OpenCorporates B: " + json.dumps(opencorp_b[:2], default=str) + "\n"
    prompt = f"""
Original A: {original_a}
Original B: {original_b}
Normalized A: {norm_a}
Normalized B: {norm_b}
Fuzzy scores: {json.dumps(scores)}

Summary of web search for A:
{summary_a}

Summary of web search for B:
{summary_b}

{oc_text}
Using the above, return JSON as requested.
"""
    return prompt

In [22]:
def query_ollama(prompt: str, model: str = "mistral"):
    try:
        resp = chat(model=model, messages=[{"role":"system","content":SYSTEM_PROMPT},
                                          {"role":"user","content":prompt}])
        text = resp["message"]["content"]  # Ollama typical shape
        return text
    except Exception as e:
        logging.exception("Ollama query failed: %s", e)
        return ""

def parse_json_safe(text: str):
    # Try to find JSON in the output
    try:
        return json.loads(text)
    except Exception:
        # crude attempt: find first { ... } substring
        m = re.search(r'{.*}', text, flags=re.DOTALL)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                return {"verdict":"ambiguous","confidence":0,"explanation":text[:400],"evidence_urls":[]}
        return {"verdict":"ambiguous","confidence":0,"explanation":text[:400],"evidence_urls":[]}

In [23]:
def summarize_hits(hits, name, model="mistral"):
    if not hits:
        return "(no useful hits)"

    hit_text = "\n".join(
        [f"{i+1}. {h.get('title','')} | {h.get('href','')} | {h.get('body','')[:300]}"
         for i, h in enumerate(hits[:5])]
    )

    system_prompt = "You are a research assistant. Summarize how these web search results relate to the firm: " + name
    prompt = f"Firm name: {name}\n\nTop web search results:\n{hit_text}\n\nGive a short summary (2-3 sentences) about what these results say about the firm."

    try:
        resp = chat(model=model, messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ])
        return resp["message"]["content"]
    except Exception as e:
        logging.warning("Failed to summarize DDG hits: %s", e)
        return "(error summarizing hits)"


In [25]:
def decide_row(a, b):
    norm_a = normalize_name(a)
    norm_b = normalize_name(b)
    scores = fuzzy_scores(norm_a, norm_b)
    result = {
        "original_a": a, "original_b": b,
        "norm_a": norm_a, "norm_b": norm_b,
        "scores": scores,
        "timestamp": datetime.utcnow().isoformat()+"Z"
    }

    # Quick decision
    if scores["token_set"] >= 95:
        result.update({"llm_verdict":"same","llm_confidence":98,"llm_explanation":"very high token_set fuzzy match","evidence_urls":[]})
        return result
    # else gather web evidence
    # hits_a = ddg_search(a, max_results=5)
    # hits_b = ddg_search(b, max_results=5)
    oc_a = opencorp_search(a, OPENCORP_TOKEN) if OPENCORP_TOKEN else []
    oc_b = opencorp_search(b, OPENCORP_TOKEN) if OPENCORP_TOKEN else []

    raw_hits_a = ddg_search(a, max_results=10)
    raw_hits_b = ddg_search(b, max_results=10)
    
    hits_a = semantic_rank(a, raw_hits_a, top_k=3)
    hits_b = semantic_rank(b, raw_hits_b, top_k=3)


    # prompt = build_user_prompt(a, b, norm_a, norm_b, scores, hits_a, hits_b, oc_a, oc_b)
    # prompt = build_user_prompt(a, b, norm_a, norm_b, scores, hits_a, hits_b)
    # Summarize DDG hits before building prompt
    ddg_summary_a = summarize_hits(hits_a, a)
    ddg_summary_b = summarize_hits(hits_b, b)
    
    prompt = build_user_prompt(
        a, b, norm_a, norm_b, scores,
        ddg_summary_a, ddg_summary_b,
        oc_a, oc_b
    )

    llm_text = query_ollama(prompt)
    llm_json = parse_json_safe(llm_text)
    result.update({
        "ddg_a": hits_a, "ddg_b": hits_b,
        # "opencorp_a": oc_a, "opencorp_b": oc_b,
        "llm_raw": llm_text,
        "llm_parsed": llm_json
    })
    return result


In [28]:
def run_all(input_csv=INPUT_CSV, output_jsonl=OUTPUT_JSONL):
    # df = df_unm2
    df = df_unm3
    out_f = open(output_jsonl, "w", encoding="utf8")
    for i, row in df.iterrows():
        a = row.get("financial_advisor_sdc", "")
        a_yr = row.get('sale_yr',9999)
        b = row.get("financial_advisor_mergent", "")
        logging.info("Processing row %d: %s | %s", i, a, b)
        res = decide_row(a, b)
        out_f.write(json.dumps(res, default=str) + "\n")
        out_f.flush()
        # polite pause
        time.sleep(1.0)
    out_f.close()
    # Save final cache state
    with open(DDG_CACHE_FILE, "wb") as f:
        pickle.dump(ddg_cache, f)


In [ ]:

if __name__ == "__main__":
    run_all()


INFO:root:Processing row 0: Harrell & Company Advisors LLC | HARRELL AND CO ADVISORS LLC
/var/folders/3s/hndc56bd6k754n21bbk34pwc0000gn/T/ipykernel_1015/4269808850.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat()+"Z"
INFO:primp:response: https://www.bing.com/search?q=Harrell+%26+Company+Advisors+LLC 200
INFO:primp:response: https://www.bing.com/search?q=Harrell+%26+Company+Advisors+LLC&first=11&FORM=PERE 200
INFO:primp:response: https://www.bing.com/search?q=HARRELL+AND+CO+ADVISORS+LLC 200
INFO:primp:response: https://www.bing.com/search?q=HARRELL+AND+CO+ADVISORS+LLC&first=11&FORM=PERE 200
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: P

In [30]:
import json

file_path = '/Users/sukanyadatta/bm_code/muni_adv_pbf/Code/results_v3.jsonl'

max_lines_to_print = 100

with open(file_path, 'r') as f:
    for line_number, line in enumerate(f, 1):
        if line_number > max_lines_to_print:
            print(f'Stopped due to max lines = {max_lines_to_print}')
            break
        else:
            line = line.strip()
            if not line:
                continue  # skip empty lines
            try:
                obj = json.loads(line)
                print(f"Line {line_number}: {obj}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {line_number}: {e}")
                # Optionally break or continue to skip corrupted line
                break  # or continue


Line 1: {'original_a': 'Harrell & Company Advisors LLC', 'original_b': 'HARRELL AND CO ADVISORS LLC', 'norm_a': 'harrell and company advisors', 'norm_b': 'harrell and co advisors', 'scores': {'ratio': 90.19607843137256, 'token_sort': 90.19607843137256, 'token_set': 93.02325581395348, 'partial': 78.26086956521739}, 'timestamp': '2025-09-27T05:14:13.516770Z', 'ddg_a': [{'title': 'Official Site of the National Hockey League | NHL.com', 'href': 'https://www.nhl.com/', 'body': "All Rights Reserved. All NHL team jerseys customized with NHL players' names and numbers are officially licensed by the NHL and …"}, {'title': 'NHL News, Scores, Stats, Standings and Rumors - National Hockey Leag…', 'href': 'https://www.cbssports.com/nhl/', 'body': 'CBS Sports has the latest NHL news, live scores, player stats, standings, fantasy games and projections on the National Hockey League.'}, {'title': 'NHL Breaking News & League Headlines | HockeyKey', 'href': 'https://hockeykey.com/news/', 'body': 'Latest 